In [85]:
import numpy as np

## Definición de la función ingresar la Matriz
Esta función se define para que el usuario pueda ingresa la matriz cuadrada y de entradas reales para la que desee encontrar valores y vectores propias

In [86]:
def ingresar_matriz():
    # Solicitar las dimensiones de la matriz
    filas = int(input("Ingresa el número de filas: "))
    columnas = int(input("Ingresa el número de columnas: "))

    print(f"Por favor, introduce las filas de la matriz con los elementos separados por espacio:")
    matriz = []

    # Recopilar las filas de la matriz
    for i in range(filas):
        while True:
            try:
                fila = list(map(float, input(f"Fila {i + 1}: ").split()))
                if len(fila) != columnas:
                    raise ValueError(f"La fila debe tener exactamente {columnas} elementos.")
                matriz.append(fila)
                break
            except ValueError as e:
                print(f"Error: {e}. Intenta de nuevo.")

    return matriz

## Ingreso de datos
se debe ingresar el numero de filas y columnas de la matriz para la que se desea encontrar los valores y vectores propios, y los valores de las entradas de la matriz ingrsadas por filas y separando cada entrada por espacios 

In [87]:
A = np.array(ingresar_matriz())
print(A)

Por favor, introduce las filas de la matriz con los elementos separados por espacio:
[[2. 1.]
 [3. 4.]]


## Definición de la cota en la que encontraremos las raices del polinomio

En la siguente casilla se calcula una cota la cota de cauchy-lagrange donde sabemos que existen todas las raices del polinomio 

In [88]:
B = 1 + max(np.abs(np.poly(A)/np.poly(A)[0])) #calculo de la cota de cauchy-lagrage para polinomios


## Creamos la función evaluar
Creamos la función evaluar para evaluar cada polinomio dado por el críterio de strums en los extremos de la partición de [-1,1] 

In [89]:
def evaluar(pol, x): #definiion de la funciòn evaluar para polinomios
  pol = pol.copy()
  n = len(pol) - 1
  return sum([pol[i] * x ** (n-i) for i in range(n+1)])

## Criterio de Strums 
creamos  la sucesion P de polinomios dados por el teorema de strums, teniendo en cuenta que el polinomio tiene n raices reales, para las raices de multiplicidad mayor a 1 este programa falla

In [90]:
#criterio de strums#
P = []#sucesión de polinomios
n = A.shape[0]#dimensiòn de la matriz por tanto numero de iteraciones de la sucesión
polinomio_original = np.poly(A)#determinante de (A-I\landa)
polinomio_polarizado = np.asarray([polinomio_original[i] * B ** (n - i) for i in range(n + 1)]) #cambio de variable#
P.append(polinomio_polarizado)#P_0 primer tèrmino de la sucesión
P.append(np.array([P[0][i] * (n-i) for i in range(0, n)]))#P_1 segundo término de la sucesión

for i in range(2, n + 1):
  P.append(-np.polynomial.polynomial.polydiv(P[i-2], P[i-1])[1])#descripción


## Conteo de raíces

Definimos esta función para contar cuantas raices existen en cada intervalo utilizando el críterio de strums

In [91]:
def conteo_raices(intervalo):
  (a, b) = intervalo
  conteo_a = 0 #conteo de los cambios de signos en cada uno de los intervalos dados por la particion en la lista dada por el teorema de strums evaluada en los limites inferiores de cada subintervalo dada por la particiones
  for i in range(1, n+1):
    if evaluar(P[i-1], a) * evaluar(P[i],a) < 0:
      conteo_a += 1

  conteo_b = 0 #conteo de los cambios de signos en cada uno de los intervalos dados por la particion
  for i in range(1,n+1):
    if evaluar(P[i-1], b)*evaluar(P[i],b) < 0:
      conteo_b += 1

  return conteo_a - conteo_b #numero de raices en los intervalos encontrados en la partición#

## Partición del intervalo 
 Estamos partiendo  el intervalo a la mitad [-1,1] en el cual parematrizamos el polinomio y vuelve a partir en la mitad los intervalos en los cuales tiene mas de una raiz contando cada cuantas raices tiene en cada sub intervalo con el criterio de strum dado por la partición

In [92]:
max_raices = conteo_raices((-1, 1))
n1 = 0
while max_raices > 1:
  k = 2 ** n1
  intervalos = [(-1 + j / k, - 1 + (j + 1) / k) for j in range(2 * k)] #particion del intervalo [-1,1]#

  max_raices = 0
  for intervalo in intervalos:

    N = conteo_raices(intervalo) #conteo de raices en cada subintervalo definido por la partción

    if N > max_raices:
      max_raices = N #localización de la raices en las particiones del intervalo[-1,1]
  n1 += 1


## Aislamiento de raíces 
Estamos creando una lista de intervalos de la partición en los cuales existen únicamente una raíz llamada "verdaderos intervalos", con la fiunción conteo la cual ingresa intervalos a la lista si la función conteo de raices en el  es de 1 si no vuelve a partirlo hasta que queden solo los intervalos que contuenen una raiz o descarta los que no tienen raices 

In [93]:
#aislamiento de raices#
verdaderos_intervalos = []
for intervalo in intervalos:
  if conteo_raices(intervalo) > 0:
    verdaderos_intervalos.append(intervalo)

## Bisección 

En esta casilla se define la función bisección que busca la uníca raiz del polinomio polarizado en los intervalos que se le den 

In [94]:
def biseccion(intervalo, tol): #Función, extremos, y tolerancia
  (a, b) = intervalo
  p = a + ( b - a ) / 2 #Punto medio del intervalo
  if evaluar(P[0], p) == 0 or b-a < tol:
    return p
  elif evaluar(P[0], a) * evaluar(P[0], p) <0:
    return biseccion((a, p),tol)
  elif evaluar(P[0], p) * evaluar(P[0], b) <0:
    return biseccion((p, b),tol)

## Encontrar las raíces del polinomio polarizado

Aquí corremos la función bisección en cada uno de los intervalos de la lista de "intervalos verdaderos" es decir busca las raices del polinomio polarizado en cada uno de los intervalos en los que sabemos que existe una única raíz

In [95]:
#encontrar las raices del polinomio parametrizado usando el método de bisección#
raices = []
for intervalo in verdaderos_intervalos:
  raices.append(biseccion(intervalo, 10**(-10)))

## Reparametrización 
Acá enontramos reparametrizamos las raíces el polinomio polarizado para  dar los valores propios de nuestro polinomio caracteristico 

In [96]:
valores_propios = [B * raiz for raiz in raices]#reparametrización de las raíces ubicadas en [-B,B]

## Definición de la función gradiente conjugado

Aca se programa el método de gradiente conjugado pues es más rápido que otros en la convergencia y dada la necesidad de usra tanto calculos y funciones anteriores es el mejor para solucionar el sistema de ecuaciones para encontrar los vectores propios aociados  cada uno de los valores propios de nuestro polinomio característico 

In [97]:
#calculo de los vectores propios por gradiente conjugado#
import numpy as np

def gradiente_conjugado(A, b, x0=None, tol=1e-10, max_iter=1000):

    n = len(b)
    x = np.zeros_like(b) if x0 is None else x0.copy()  # Vector inicial (usualmente 0)
    r = b - np.dot(A, x)  # Resíduo inicial
    p = r.copy()  # Dirección de búsqueda inicial

    for k in range(max_iter):
        # Producto escalar de p con r using .flatten() to ensure 1D arrays for dot product
        # This ensures the dot product calculates a scalar as intended.
        alpha = np.dot(r.flatten(), r.flatten()) / np.dot(p.flatten(), np.dot(A, p).flatten())

        # Actualización de la solución
        x = x + alpha * p

        # Nuevo residuo
        r_new = r - alpha * np.dot(A, p)

        # Comprobar la convergencia
        if np.linalg.norm(r_new) < tol:
            print(f"Convergió después de {k+1} iteraciones.")
            return x

        # Calculamos beta using .flatten() to ensure 1D arrays for dot product
        beta = np.dot(r_new.flatten(), r_new.flatten()) / np.dot(r.flatten(), r.flatten())

        # Actualizamos la dirección
        p = r_new + beta * p

        r = r_new  # Actualizamos el residuo

    print("Alcanzado el número máximo de iteraciones.")
    return x

## Vectores propios

En la siguiente casilla se ejecutan las funciones anteriores para para encontrar los vectores propios asociado a cada uno de los valores propios asociados y reporta los valores y vectores propios asociados a la matriz ingresada

In [98]:
#solucion del sistema de ecuaciones para cada valor propio
for valor in valores_propios:
  #sistema de ecuaciones para cada valor propio
  A_hat = A - valor * np.eye(n) #matriz (A-\landa I)
  b = np.zeros((n,1)) #vector 0 de dimension nx1
  x0 = np.ones((n,1))#valor de inicio para el método gauss seidel en este caso la identidad
  x = gradiente_conjugado(A_hat,b,x0, 1e-4,100)#solución del sistema de ecuaciones (A-\landa I)x=0
  print(f"El autovector asociado a {valor} es {x}")

Convergió después de 1 iteraciones.
El autovector asociado a 1.0000000000873115 es [[ 0.5]
 [-0.5]]
Convergió después de 1 iteraciones.
El autovector asociado a 5.000000000029104 es [[0.5]
 [1.5]]
